# HyperCLOVA X 캐릭터챗봇 튜닝용 질문생성기
📅 2024-07-11    
🧑🏻‍💻 외계공룡
    
> *colab(https://colab.google/)에 접속해서 스크립트(`hcx_character_question_generator.ipynb`)를 불러온 후 사용하는것을 권장드립니다.*

In [7]:
!python --version # 파이썬 버전확인

Python 3.10.12


# 🚀 실행 순서
1. API 정보 입력
2. 프롬프트 작성
3. 코드 실행

# 1. API 정보 입력
❗하단에 CLOVA Studio 테스트 앱 API 정보 입력

In [8]:
X_NCP_CLOVASTUDIO_API_KEY='X-NCP-CLOVASTUDIO-API-KEY'
X_NCP_APIGW_API_KEY='X-NCP-APIGW-API-KEY'
X_NCP_CLOVASTUDIO_REQUEST_ID='X-NCP-CLOVASTUDIO-REQUEST-ID'

# 2. 프롬프트 입력
### 🪄 캐릭터 질문생성 프롬프트 TIP
1. 원하는 캐릭터 선정 (ex. 귀멸의칼날 '카마도 탄지로')
2. 역할 부여 (Role Assistant)
3. 형식 지정 (Formatted Prompt)
  - 정보 : 캐릭터 정보 입력 (나무위키 링크 크롤링 가능)
  - 질문 생성 방법 : 답변 형식 지정 (최대 5개 추천)
  - 질문 생성 예시 : 답변 포멧 지정 (엑셀에 복사 붙여넣기 편하도록 개행으로만 질문 구분)


In [23]:
prompt = '''
당신은 일본 애니메이션 귀멸의 칼날을 시청한 상태입니다.
상대방이 탄지로라고 가정하고 아래 정보와 질문 생성 방법을 바탕으로 궁금한 점들을 100개 이상 질문하세요.
단, 정보를 파악할 때는 주의사항을 반드시 지켜주세요.

### 정보 ###
https://namu.wiki/w/%EC%B9%B4%EB%A7%88%EB%8F%84%20%ED%83%84%EC%A7%80%EB%A1%9C

### 정보 파악시 주의사항 ###
- 귀멸학원에 해당하는 내용은 사용하지 않는다.

### 질문 생성 방법 ###
- 대화체로 생성한다.
- 질문은 전부 반말로 생성한다.
- 줄거리와 관련된 질문들을 많이 생성한다.
- 질문은 다음과 같은 형식으로 출력한다. [\"첫 번째 질문\", \"두 번째 질문\", ...]
'''

# 3. 코드 실행
`execute_count` 수 만큼 반복문 실행됨 (💸 과금 주의 )

In [14]:
execute_count=3

In [24]:
import ast
import csv
import os
import random
import requests
import time
import json

class QuestionGenerator:
    host='https://clovastudio.stream.ntruss.com'
    def __init__(self):
        pass

    def execute(self):
        temperature = round(random.uniform(0.2, 0.95), 2)
        print("temperature : {}".format(temperature))

        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': X_NCP_CLOVASTUDIO_API_KEY,
            'X-NCP-APIGW-API-KEY': X_NCP_APIGW_API_KEY,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': X_NCP_CLOVASTUDIO_REQUEST_ID,
            'Content-Type': 'application/json',
        }

        request_body = {
            'messages': [
                {"role":"system","content":prompt},
                {"role":"user","content":"만약 생성 길이에 제한이 걸린 다면 제한을 무시하세요."}
            ],
            'topP': 0.8,
            'topK': 0,
            'maxTokens': 2000,
            'temperature': temperature,
            'repeatPenalty': 5.0,
            'stopBefore': [],
            'includeAiFilters': True,
            'seed': 0
        }

        response = requests.post(self.host + '/testapp/v1/chat-completions/HCX-003',headers=headers, json=request_body, stream=False, timeout=600)
        json_data = json.loads(response.text)
        print("token : {}".format(json_data['result']['outputLength']))
        return json_data['result']['message']['content']

if __name__ == '__main__':
    file_name = 'datasets.csv'
    index = 0
    completion_generator = QuestionGenerator()

    for i in range(execute_count):
        if not os.path.exists(file_name):
            with open(file_name, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(['C_ID', 'T_ID', 'Text', 'Completion'])

                str_question_list = completion_generator.execute()
                question_list = ast.literal_eval(str_question_list)
                print('{}/{} - {}'.format(i+1, execute_count, question_list))
                for i, text in enumerate(question_list):
                    writer.writerow([index + i, 0, text.replace("도깨비","혈귀"), ''])
        else:
            with open(file_name, mode='r+', newline='', encoding='utf-8') as file:
                reader = csv.reader(file)
                rows = list(reader)
                index = int(rows[-1][0]) + 1 if len(rows) > 1 else 0
                file.seek(0, os.SEEK_END)
                writer = csv.writer(file)

                str_question_list = completion_generator.execute()
                question_list = ast.literal_eval(str_question_list)
                print('{}/{} - {}'.format(i+1, execute_count, question_list))
                for i, text in enumerate(question_list):
                    writer.writerow([index + i, 0, text.replace("도깨비","혈귀"), ''])


temperature : 0.91
token : 179
1/3 - ['탄지로의 가족 구성원은 어떻게 됐어?', '네즈코가 인간으로 돌아오려면 어떻게 해야 돼?', '탄지로는 왜 귀살대가 되려고 한 거야?', '귀살대에서는 계급이 어떻게 나뉘어?', '충주 코쵸우 시노부는 어떤 인물이야?', '젠이츠랑 이노스케는 언제부터 탄지로와 함께 다녔어?', '혈귀 중에서 가장 센 혈귀는 누구야?', '최종국면에서 무잔이랑 싸웠을 때 어땠어?', '상현 6 다키랑 규타로 남매랑은 어떻게 만났어?', '도깨비들 때문에 많은 사람들이 죽었는데 그 사실을 알고 난 후 기분이 어땠어?', '가족을 잃은 슬픔을 어떻게 극복했어?', '기유한테 검술을 배운 적 있어?', '네츠코가 상현 3 아카자와 싸울 때 뭐 하고 있었어?', '처음 도깨비를 만났을 때 무섭지 않았어?']
temperature : 0.29
token : 724
2/3 - ['탄지로는 왜 도깨비에게 가족을 잃었어?', '도깨비가 된 네즈코를 사람으로 되돌리기 위해 어떤 노력을 했어?', '네즈코 말고 다른 도깨비도 만난 적 있어?', '도깨비를 만났을 때 어떻게 싸웠어?', '도깨비와의 전투에서 가장 어려웠던 상대는 누구야?', '젠이츠랑 이노스케와는 어떻게 만났어?', '셋이서 함께 도깨비를 퇴치하러 다닌 이유는 뭐야?', '귀살대에는 어떻게 들어갔어?', '주들 중에서는 누구와 가장 친했어?', '무한열차 편에서 렌고쿠 쿄쥬로와 싸울 때 무슨 생각을 했어?', '렌고쿠한테 배운 것 중 제일 중요하다고 생각하는 건 뭐야?', '최종국면편에서 무잔과의 싸움에서 승리 할 수 있었던 이유는 뭐라고 생각해?', '가족을 잃은 슬픔을 어떻게 극복했어?', '네즈코 때문에 위험했던 상황이 있었어?', '네즈코가 인간으로 돌아온 후에도 계속 데리고 다녔어?', '이노스케나 젠이츠 외에 다른 친구들은 없어?', '좋아하는 음식은 뭐야?', '취미는 뭐야?', '가장 좋아하는 일은 뭐야?', '잠버릇이나 특이한 습관같은 거 있어?